In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np

load_dotenv()

# Get the API key from the environment
API_KEY = os.getenv("STEAM_API_KEY")
MARKET_API_KEY = os.getenv("STEAM_MARKET_API_KEY")

In [2]:
steam_store = "https://store.steampowered.com"
# /appreviews/10500?
base_params = {
    "json": 1,
    "language": "english",
    "filter": "recent",
    "num_per_page": 100,
    "cursor": "*",
}

In [3]:
steam_market = 'https://api.steamapis.com'
app_list_url = f"{steam_market}/market/apps?"
market_params = {
    "api_key": MARKET_API_KEY
}

In [6]:
all_games = requests.get(app_list_url, params=market_params).json()

In [7]:
all_games_df = pd.DataFrame(all_games)

In [8]:
all_games_df.head()

,appID,name,is_free,price_overview
0,10500,Total War: EMPIRE – Definitive Edition,False,"{'final_formatted': '$24.99', 'initial_formatt..."
1,391070,Stellar 2D,False,"{'final_formatted': '$0.99', 'initial_formatte..."
2,576670,Sky Is Arrows,False,"{'final_formatted': '$9.99', 'initial_formatte..."
3,256410,Might & Magic: Duel of Champions,True,NaN
4,546330,SPACE MOUSE 35th Anniversary edition,False,"{'final_formatted': '$4.99', 'initial_formatte..."


In [10]:
all_games_df.to_csv("data/steam/all_games.csv", index=False)

In [2]:
all_games_df = pd.read_csv("data/steam/all_games.csv")

In [5]:
all_games_df.head()

,appID,name,is_free,price_overview
0,10500,Total War: EMPIRE – Definitive Edition,False,"{'final_formatted': '$24.99', 'initial_formatt..."
1,391070,Stellar 2D,False,"{'final_formatted': '$0.99', 'initial_formatte..."
2,576670,Sky Is Arrows,False,"{'final_formatted': '$9.99', 'initial_formatte..."
3,256410,Might & Magic: Duel of Champions,True,NaN
4,546330,SPACE MOUSE 35th Anniversary edition,False,"{'final_formatted': '$4.99', 'initial_formatte..."


In [3]:
game_reviews = pd.read_csv("data/steam/long_reviews.csv")

In [7]:
reviews_from_api = []

last_review = game_reviews.tail(1)["name"].values[0]

# offset = 0
offset = all_games_df[all_games_df['name'] == last_review].index[0]

In [13]:
remaining_games = all_games_df.iloc[offset:]

In [14]:
import urllib

for i, app_id in enumerate(remaining_games["appID"]):
    cursor = '*'
    reviews_url = f"{steam_store}/appreviews/{app_id}?"
    past_cursors = []

    positive = False

    print(f"Getting reviews for {app_id} ({i+1}/{len(remaining_games)})", end="\r")

    while cursor:
        params = base_params.copy()
        params["cursor"] = cursor
        params["review_type"] = 'positive' if positive else "negative"
        url = f"{reviews_url}"\
        + f"json=1&language=english&filter=recent&num_per_page=100&"\
        + f"cursor={cursor}&review_type={'positive' if positive else 'negative'}"
        # print(url)
        res = requests.get(url)
        try:
            reviews = res.json()
        except:
            try:
                with open(res, encoding='utf-8-sig') as r:
                    reviews = r.json()
            except:
                print('Unable to process response')
                break
        cursor = reviews.get("cursor")

        if cursor:
            cursor = urllib.parse.quote(cursor)
        if cursor in past_cursors:
            if positive:
                break
            positive = True
            cursor = '*'
            past_cursors = []
            continue
        if not cursor:
            if positive:
                break
            positive = True
            cursor = '*'
            past_cursors = []
            continue
        past_cursors.append(cursor)

        if cursor:
            for review in reviews["reviews"]:
                author = review["author"]["steamid"]

                reviews_from_api.append({
                    "app_id": app_id,
                    "author": author,
                    "recommended": positive
                })
        else:
            break

# sort reviews by review_weighted
reviews_df = pd.DataFrame(reviews_from_api)

# set max number of rows to none
pd.set_option('display.max_rows', None)

reviews_df.tail()


,app_id,author,recommended
6946148,997070,76561198053561661,True
6946149,997070,76561198030558053,True
6946150,997070,76561198068534122,True
6946151,997070,76561197983376650,True
6946152,997070,76561198230763904,True


In [71]:
reviews_df = pd.DataFrame(reviews_from_api)

In [15]:
name_id = pd.DataFrame(all_games_df[['appID', 'name']])
name_id = name_id.rename(columns={'appID': 'app_id'})

In [16]:
reviews_df = reviews_df.merge(name_id, on='app_id')
display(reviews_df.head(2))
display(reviews_df.tail(2))

,app_id,author,recommended,name
0,976730,76561197978030555,False,Halo: The Master Chief Collection
1,976730,76561198001410886,False,Halo: The Master Chief Collection


,app_id,author,recommended,name
6956117,997070,76561197983376650,True,Marvel's Avengers
6956118,997070,76561198230763904,True,Marvel's Avengers


In [18]:
reviews_df.drop(columns=['app_id'], inplace=True)

In [7]:
display(reviews_df.head(2))
display(reviews_df.tail(2))

NameError: name 'reviews_df' is not defined

In [9]:
display(game_reviews.head())
display(game_reviews.tail())

,author,recommended,name
0,76561198140439494,False,Total War: EMPIRE – Definitive Edition
1,76561198001149578,False,Total War: EMPIRE – Definitive Edition
2,76561199667392107,False,Total War: EMPIRE – Definitive Edition
3,76561198024411821,False,Total War: EMPIRE – Definitive Edition
4,76561198297284962,False,Total War: EMPIRE – Definitive Edition


,author,recommended,name
19596706,76561198053561661,True,Marvel's Avengers
19596707,76561198030558053,True,Marvel's Avengers
19596708,76561198068534122,True,Marvel's Avengers
19596709,76561197983376650,True,Marvel's Avengers
19596710,76561198230763904,True,Marvel's Avengers


In [20]:
# add the new reviews to the existing reviews

merged_reviews = pd.concat([game_reviews, reviews_df])

In [75]:
len(reviews)

4

In [21]:
len(merged_reviews)

19634690

In [22]:
merged_reviews.duplicated().sum()

37979

In [82]:
merged_reviews.drop(columns=['app_id'], inplace=True)

In [23]:
game_reviews = merged_reviews.drop_duplicates()
game_reviews.to_csv("data/steam/long_reviews.csv", index=False)

In [18]:
display(len(reviews_df))
display(reviews_df.duplicated().sum())
reviews_df.drop_duplicates(inplace=True)
display(reviews_df.duplicated().sum())

display(len(reviews_df))

1000

0

0

1000

In [19]:
print(reviews_df.head())
print(reviews_df.tail())

   app_id             author  recommended
0   10500  76561199100623571        False
1   10500  76561198140439494        False
2   10500  76561198001149578        False
3   10500  76561199667392107        False
4   10500  76561198024411821        False
     app_id             author  recommended
995   10500  76561197991580763        False
996   10500  76561198120223099        False
997   10500  76561197970772540        False
998   10500  76561198170133919        False
999   10500  76561198152676419        False


In [2]:
game_reviews = pd.read_csv("data/steam/long_reviews.csv")

In [2]:
reviews = pd.read_csv("data/steam/long_reviews.csv")

last_review = reviews.tail(1)["name"].values[0]

In [3]:
last_review

"Marvel's Avengers"

In [3]:
# find drop any author that has less than 2 reviews
author_counts = game_reviews['author'].value_counts()
author_counts = author_counts[author_counts >= 5]

gr_no_dup = game_reviews[game_reviews['author'].isin(author_counts.index)]

len(gr_no_dup)

7114866

In [4]:
gr_no_dup['recommended'] = gr_no_dup['recommended'].apply(lambda x: 1 if x else -1)

gr_no_dup.head()

C:\Users\Tim\AppData\Local\Temp\ipykernel_25328\2831849930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gr_no_dup['recommended'] = gr_no_dup['recommended'].apply(lambda x: 1 if x else -1)


,author,recommended,name
1,76561198001149578,-1,Total War: EMPIRE – Definitive Edition
4,76561198297284962,-1,Total War: EMPIRE – Definitive Edition
5,76561198039423947,-1,Total War: EMPIRE – Definitive Edition
11,76561198089482980,-1,Total War: EMPIRE – Definitive Edition
17,76561197997373898,-1,Total War: EMPIRE – Definitive Edition


In [7]:
gr_no_dup['recommended'].value_counts()

 1    5928675
-1     962505
Name: recommended, dtype: int64

Run the following 2 cells until the number ceases to change.

In [19]:
# delete games with less than 50 reviews
game_counts = gr_no_dup['name'].value_counts()
game_counts = game_counts[game_counts >= 50]

gr_no_dup = gr_no_dup[gr_no_dup['name'].isin(game_counts.index)]


In [20]:
author_counts = gr_no_dup['author'].value_counts()
author_counts = author_counts[author_counts >= 5]

gr_no_dup = gr_no_dup[gr_no_dup['author'].isin(author_counts.index)]

len(gr_no_dup)

7008489

In [6]:
review_count = 0

MIN_GAME_REVIEWS = 100
MIN_AUTHOR_REVIEWS = 5

while len(gr_no_dup) != review_count:
    review_count = len(gr_no_dup)
    # drop games with less than 50 reviews
    game_counts = gr_no_dup['name'].value_counts()
    game_counts = game_counts[game_counts >= MIN_GAME_REVIEWS]

    gr_no_dup = gr_no_dup[gr_no_dup['name'].isin(game_counts.index)]

    # drop authors with less than 5 reviews
    author_counts = gr_no_dup['author'].value_counts()
    author_counts = author_counts[author_counts >= MIN_AUTHOR_REVIEWS]

    gr_no_dup = gr_no_dup[gr_no_dup['author'].isin(author_counts.index)]

In [30]:
store.close()

In [8]:
import threading

NUM_THREADS = 20

def pivot_reviews(df, results, i):
    print(f"Thread {i} started")
    pivot = df.pivot_table(index='name', columns='author', values='recommended').fillna(0)
    print(f"Thread {i} finished")
    results.append(pivot)
    return

def pivot_reviews_threaded(df):
    threads = []
    results = []
    chunk_size = len(df) // NUM_THREADS
    chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

    for i, chunk in enumerate(chunks):
        t = threading.Thread(target=pivot_reviews, args=(chunk, results, i))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    # get all the responses from the treads

    return pd.concat([r for r in results]).fillna(0)

In [25]:
game_pivot.to_csv("data/steam/game_pivot.csv")

In [9]:
game_pivot = pivot_reviews_threaded(gr_no_dup)

Thread 0 started
Thread 1 started
Thread 2 started
Thread 3 started
Thread 4 started
Thread 5 started
Thread 6 started
Thread 7 started
Thread 8 started
Thread 9 started
Thread 10 started
Thread 11 started
Thread 12 started
Thread 13 started
Thread 14 started
Thread 15 started
Thread 16 started
Thread 17 started
Thread 18 started
Thread 19 started
Thread 0 finished
Thread 6 finished
Thread 3 finished
Thread 2 finished
Thread 1 finished
Thread 9 finished
Thread 4 finished
Thread 11 finished
Thread 5 finished
Thread 14 finished
Thread 7 finished
Thread 18 finished
Thread 17 finished
Thread 10 finished
Thread 13 finished
Thread 16 finished
Thread 19 finished
Thread 12 finished
Thread 8 finished
Thread 15 finished


In [18]:
# game_pivot.fillna(0, inplace=True)

# display(game_pivot.describe())
display(game_pivot.shape)

# rotate the pivot table
rot_pivot = game_pivot.T
display(rot_pivot.shape)

# write the pivot table to an xl file
rot_pivot.to_excel("data/steam/rot_pivot.xlsx")

(3541, 744562)

(744562, 3541)

In [10]:
game_pivot.fillna(0, inplace=True)
display(game_pivot.head(3))
display(game_pivot.tail(3))



author,76561197960267984,76561197960268765,76561197960269230,76561197960269409,76561197960269579,76561197960269906,76561197960270054,76561197960270526,76561197960270999,76561197960271098,...,76561198070882733,76561198105301758,76561198106538641,76561198134309424,76561198145028927,76561198150849751,76561198156238166,76561198797517897,76561199384271546,76561198023504801
name,,,,,,,,,,,,,,,,,,,,,
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arma Tactics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BIT.TRIP RUNNER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


author,76561197960267984,76561197960268765,76561197960269230,76561197960269409,76561197960269579,76561197960269906,76561197960270054,76561197960270526,76561197960270999,76561197960271098,...,76561198070882733,76561198105301758,76561198106538641,76561198134309424,76561198145028927,76561198150849751,76561198156238166,76561198797517897,76561199384271546,76561198023504801
name,,,,,,,,,,,,,,,,,,,,,
klocki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
planetarian HD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
永遠消失的幻想鄉 ～ The Disappearing of Gensokyo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# create a sparse matrix using game_pivot
from scipy.sparse import csr_matrix
game_sparse = csr_matrix(game_pivot)


In [13]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors( algorithm='brute', n_neighbors=10)
knn.fit(game_sparse)

NearestNeighbors(algorithm='brute', n_neighbors=10)

In [14]:
import pickle

with open("data/steam/knn.pkl", "wb") as f:
    pickle.dump(knn, f)

In [16]:
import pyarrow as pa
import pyarrow.parquet as pq

pq.write_table(pa.Table.from_pandas(game_pivot), 'data/steam/game_pivot.parquet')

TypeError: __cinit__() got an unexpected keyword argument 'thrift_compression'